## Rendu Neo4J

Luca Delanglade - Nicolas Lamy


Notre étude porte sur les livres prêtés en bibliothèques à Paris: <br>https://opendata.paris.fr/explore/dataset/tous-les-documents-des-bibliotheques-de-pret/information/ <br> Notre dataset contient les informations relatives au pret de chaque document disponible à l'emprunt et comptabilise le nombre de prets enregistrés sur l'année 2017. Nous avons souhaité le compléter avec des données accesibles sur l'API Googlebook.

Notre travail s'organise autour de 3 notebook: <br>
-notebook Data Preprocessing -- traitements éffectués en amont pour l'import sur Neo4J <br>
-notebook Data Visualisation -- analyser la distribution des données et restreindre l'étude <br>
-notebook Googlebook scrapping -- permet d'ajouter une colonne de description des livres au dataset <br>


Nous avons restreint notre étude à la catégorie litterature, qui comptabilise le plus grand nombre de prets à l'année : 200 000. Etant confronté à une limitation de 1000 requetes par jour via l'API Googlebook, nous avons importé les descriptions par batch de 1000.

In [85]:
import pandas as pd
data_neo4J=pd.read_csv("data_litterature.csv").drop(['Unnamed: 0'],axis=1)
data_neo4J.head()

,Titre,sstitre,Auteur,Editeur,Date,prets,Nombre de prêts par exemplaire,exemplaires,localisations,Langue,cat,tauxemprunt
0,Chanson douce,,Leïla Slimani,Gallimard,2016,1169.0,16.236111,72.0,51.0,français,LFRA Litterature francaise,68.0
1,Petit pays,,Gaël Faye,Bernard Grasset,2016,889.0,14.338710,62.0,47.0,français,LFRA Litterature francaise,60.0
2,Sur les chemins noirs,,Sylvain Tesson,Gallimard,2016,856.0,16.150943,53.0,49.0,français,LFRA Litterature francaise,67.0
3,Arrête avec tes mensonges,,Philippe Besson,Julliard,2017,846.0,16.588235,51.0,49.0,français,LFRA Litterature francaise,69.0
4,Article 353 du code pénal,,Tanguy Viel,les Éditions de Minuit,2017,785.0,14.811321,53.0,52.0,français,LFRA Litterature francaise,62.0


### Objectifs

1: Nous souhaitons développer un outil de visualisation de type graphe permettant au public de faire une recherche sur la base du titre ou de l'auteur. La visualisation permet de renseigner le public sur la difficulté qu'il aura à emprunter chaque ouvrage et donnera accès à la description de l'ouvrage présente sur Googlebook.

2: Nous souhaitons préconiser des solutions destinées aux bibliothèques afin de répondre au mieux aux besoins de leurs lecteurs en agissant sur le nombre de localisations et sur le nombre d'exemplaires de chaque ouvrage. Pour ce faire, nous allons utiliser des algorithmes non supervisés qui permettent de catégoriser chaque ouvrage en fonction du nombre d'exemplaires, du nombre de localisations et du nombre de prets.

## Import

In [86]:
from neo4j import GraphDatabase

DB_URI = "bolt://localhost:7687"
DB_USER = "neo4j"
DB_PASSWORD = "0207"
driver = GraphDatabase.driver(DB_URI, auth=(DB_USER, DB_PASSWORD))

Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("10054; 'Une connexion existante a dû être fermée par l’hôte distant'; None; 10054; None")
Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("10054; 'Une connexion existante a dû être fermée par l’hôte distant'; None; 10054; None")


In [87]:
#Contraintes
CONTRAINTES=["CREATE CONSTRAINT ON (l:Livre) ASSERT l.titre IS UNIQUE;", 
"CREATE INDEX ON :Auteur(nomauteur);",
"CREATE INDEX ON :Editeur(nomediteur);",
"CREATE INDEX ON :Categorie(nomcat);"]
for i in CONTRAINTES:
    with driver.session() as session:
        session.run(i)

In [89]:
#Import des données principales
IMPORT1="""
USING PERIODIC COMMIT 500 

LOAD CSV WITH HEADERS FROM 'file:///data_litterature.csv' 
AS ligne with ligne where ligne.Titre is not null

   MERGE (l:Livre {titre: trim(ligne.Titre)}) 

   ON CREATE SET l.soustitre = trim(ligne.sstitre) 
   ON CREATE SET l.date = ToInteger(ligne.Date) 
   ON CREATE SET l.prets = ToInteger(ligne.prets) 
   ON CREATE SET l.exemplaires = ToInteger(ligne.exemplaires)
   ON CREATE SET l.localisations = ToInteger(ligne.localisations)
   ON CREATE SET l.tauxemprunt = ToInteger(ligne.tauxemprunt)
   ON CREATE SET l.langue = trim(ligne.Langue)
   
   MERGE (auteur:Auteur { nomauteur: trim(ligne.Auteur) }) 
   
   MERGE (edi:Editeur { nomediteur: trim(ligne.Editeur) })
   
   MERGE (cat:Categorie { nomcat: trim(ligne.cat)}) 
   
   MERGE (l)-[: ECRIT]->(auteur)
   MERGE (l)-[: EDITE]->(edi) 
   MERGE (l)-[: APPARTIENT]->(cat) ;

"""
with driver.session() as session:
    session.run(IMPORT1)

In [91]:
#Import des données de GoogleBook
IMPORT2="""

LOAD CSV WITH HEADERS FROM 'file:///Googlebook_db.csv' 
AS ligne with ligne where ligne.Titre is not null

   MERGE (l:Livre {titre: trim(ligne.Titre)}) SET l.summary = trim(ligne.Summary)
   
"""
with driver.session() as session:
    session.run(IMPORT2)

## REQUETES

In [96]:
#Qui sont les auteurs américains les plus lus?
REQUETE1="""

MATCH (cat)--(l:Livre)--(a:Auteur)
MATCH (l:Livre)--(e:Editeur)
WHERE left(cat.nomcat,4)="LNAM"
RETURN distinct(l.titre),a.nomauteur,e.nomediteur, l.langue, l.prets
ORDER BY l.prets DESC
LIMIT 20

"""
with driver.session() as session:
    result=session.run(REQUETE1)
    df = pd.DataFrame(result, columns=["Titre","Auteur","Editeur","Langue","Prets"])
df.head(10)

,Titre,Auteur,Editeur,Langue,Prets
0,Intimidation,Harlan Coben,Feryane,français,667
1,Intimidation,Harlan Coben,Belfond,français,667
2,Intimidation,Harlan Coben,Pocket,français,667
3,Message sans réponse,Patricia J. MacDonald,Albin Michel,français,597
4,Message sans réponse,Patricia J. MacDonald,Editions Libra diffusio,français,597
5,La vengeance des mères,Jim Fergus,Cherche midi,français,517
6,La vengeance des mères,Jim Fergus,A vue d'oeil,français,517
7,La vengeance des mères,Jim Fergus,Pocket,français,517
8,Le piège de la Belle au bois dormant,Mary Higgins Clark,Albin Michel,français,495
9,Une avalanche de conséquences,Elizabeth George,Pocket,français,495


In [97]:
#Quels sont les Livres américains les plus difficile à emprunter?
REQUETE2="""

MATCH (cat)--(l:Livre)--(a:Auteur)
MATCH (l:Livre)--(e:Editeur)
WHERE left(cat.nomcat,4)="LNAM"
RETURN l.titre,a.nomauteur,e.nomediteur, l.langue, l.tauxemprunt
ORDER BY l.tauxemprunt DESC
LIMIT 20

"""
with driver.session() as session:
    result=session.run(REQUETE2)
    df = pd.DataFrame(result, columns=["Titre","Auteur","Editeur","Langue","Tauxemprunt"])
df.head(10)

,Titre,Auteur,Editeur,Langue,Tauxemprunt
0,Ne pleure pas,Mary Kubica,HarperCollins,français,85
1,Sphinx,Anne Garréta,Librairie Générale Française,français,75
2,Sphinx,Nicolas Jarry,Éd. du Rocher,français,75
3,Sphinx,Anne Garréta,Éd. du Rocher,français,75
4,Sphinx,Robin Cook,B. Grasset,français,75
5,Sphinx,Robin Cook,Librairie Générale Française,français,75
6,Sphinx,Christine Falkenland,Librairie Générale Française,français,75
7,Sphinx,Nicolas Jarry,B. Grasset,français,75
8,Sphinx,Christine Falkenland,Éd. du Rocher,français,75
9,Sphinx,Christine Falkenland,Actes Sud,français,75


In [98]:
#Nous souhaitons emprunter plusieurs livres d'un même auteur
REQUETE3="""

MATCH (l:Livre)--(a:Auteur)
WITH a, count(distinct(l.titre)) as rel, collect(l.titre) as l
WHERE rel > 1
RETURN a.nomauteur, rel, l
ORDER BY rel DESC
LIMIT 20

"""
pd.set_option('max_colwidth', 300)
with driver.session() as session:
    result=session.run(REQUETE3)
    df = pd.DataFrame(result,columns=["Auteur","Nombre d'ouvrages","Titres"])
 
df.head(10)

,Auteur,Nombre d'ouvrages,Titres
0,Georges Simenon,210,"[Mémoires intimes, L'Île des maudits, L'oeil de l'Utah, Marins pour rire, marins quand même, Maigret en Auvergne, La Jument-Perdue, Les grandes enquêtes de Maigret, Lettres à ma mère, La Fiancée du diable, La folle de Maigret, Portrait-souvenir de Balzac, Les 13 coupables, La maison de l'inquiét..."
1,Agatha Christie,202,"[Les enquêtes de miss Marple, 4.50 from Paddington, 1920s Omnibus, 0Œuvres complètes, Miss Marple Omnibus, Meurtres au soleil, Miss Marple and mystery, Oeuvres complètes, Le Cheval à bascule..., A daughter’s a daughter and other novels, Murder of Roger Ackroyd, Muoi nguoi da den nho, Cáng shū sh..."
2,Honoré de Balzac,148,"[Xiju, Théâtre, Miêng da lua, Journaux à la mer, Histoire véritable de la bossue courageuse, Contes bruns, par une tête à l'envers, Fenxi yanjiu, Les fantaisies de la Gina, Russie-Express, Les petits bourgeois, Physiologie de l'employé, La Comédie Humaine, Des salons littéraires et des mots élog..."
3,James Patterson,140,"[Jack and Jill, Et tombent les filles, 15e affaire, Black friday, Cat and mouse, 9th judgement, Lettres de sang, Toys, Sam's letters to Jennifer, 4th of July, When the wind blows, Step on a crack, L'été des machettes, Cradle and all, Swimsuit, Fialki sinie, Med head, Judge ang jury, Murder house..."
4,Alexandre Dumas,136,"[Histoire d'un casse-noisette, Othon l'archer, Histoire de mes bêtes, Les poules de M. de Chateaubriand, Une aventure d'amour, La peine de mort, Le docteur mystérieux, Souvenirs dramatiques, Robin des Bois, Sur Gérard de Nerval..., Les morts vont vite, Quinze jours au Sinaï, Chá huā nǚ, Histoire..."
5,Danielle Steel,134,"[La ronde des souvenirs, Star, Souvenirs du Vietnam, Khamsat ayyâm fî bârîs, Al-manzil, Marfa' al-amân, Rogue, The wedding, Honneur et courage, Un pur bonheur, Au nom du cœur, Mirror image, al-zifâf, Accident, Collection privée, Le fantô̂me, Daqqat qalb, Cher daddy, Honor thyself, Loving, Malvei..."
6,Henry James,132,"[Impressions anglaises, 0Lettres à sa famille, La situation littéraire actuelle en France, Le tour d'ecrou, La création littéraire, Les Fantômes de la jalousie, Daisy Miller ; and, the turn of the screw, La madone de l'avenir, La scène américaine, Le Dernier des Valerii, Rêves yankees, In extrem..."
7,Stephen King,130,"[Colorado Kid, Pet sematary, Lovec snov, La ballade de la balle élastique, Koldun i kristall, Sleeping beauties, 'Salem's Lot, Transgressions, Mobilʹnik, Chantier, Everything's eventual, On writing, Tma, - i bolshe nichego, Hei an de ling yi ban, Dead zone (L'accident), Wolves of the calla, Duma..."
8,Anne Perry,121,"[An echo of murder, Angels in the gloom, Un innocent à l'Old Bailey, The one thing more, No graves as yet, The silent cry, Cardington Crescent, Les deux premières enquêtes de William Monk, Farrier's Lane, Callander Square, Utoplennik iz Bljugejt-filds, Vor s Rutlend-plejs, Slaves and Obsession, ..."
9,Victor Hugo,117,"[Chansons de Victor Hugo, Journal de ce que j'apprends chaque jour, Les Sept cordes, Epitres, Avant l'exil, Quatre vingt-treize, Les Sept cordes ; La Corde d'airain, Actes et paroles, Correspondance entre Victor Hugo et Pierre-Jules Hetzel, Deuxieme partie, Premiere partie, Bug-Jargal, Actes et ..."


In [99]:
#Nous souhaitons voir quels sont les derniers livres édités par Gallimard qui sont disponibles à l'emprunt
REQUETE4="""

MATCH (e:Editeur{nomediteur:"Gallimard"})--(l:Livre)--(a:Auteur)
WHERE l.date IS NOT NULL
RETURN l.titre,a.nomauteur, l.date, l.tauxemprunt
ORDER BY l.date DESC
LIMIT 20

"""

with driver.session() as session:
    result=session.run(REQUETE4)
    df = pd.DataFrame(result, columns=["Titre","Auteur","Date","Taux d'emprunt"])
 
df.head(10)

,Titre,Auteur,Date,Taux d'emprunt
0,Le coeur content,Nanoucha Van Moerkerkenland,2018,0
1,SS-GB,Len Deighton,2018,0
2,La grande roue,Diane Peylin,2018,0
3,"Erri de Luca, entre Naples et la Bible",Henri Godard,2018,0
4,Lynwood Miller,Sandrine Roy,2018,0
5,Casse-gueule,Clarisse Gorokhoff,2018,0
6,La grande roue,Vaclav Havel,2018,0
7,Quelle n'est pas ma joie,Jens Christian Grondahl,2018,0
8,Un si beau diplôme !,Scholastique Mukasonga,2018,0
9,"Et moi, je vis toujours",Jean d' Ormesson,2018,0
